In [16]:
import json
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import pickle   
from lifelines import CoxPHFitter # install lifelines-0.30.0 if not already installed
from lifelines.utils import concordance_index


In [17]:
# load test features
test_radiomics = pd.read_csv('D:\\Shruti\\IS_project\\test_radiomics_MN.csv')

# retain patient_id separately; you need the surival data for these patients also (survival time and event columns)
pids = test_radiomics['patient_id'].values
# survival_time = test_radiomics['survival_time'].values
# event = test_radiomics['event'].values
test_radiomics = test_radiomics.drop(columns=['patient_id'])

# remove features that dont have 'original' in their name
# keep_cols = [col for col in test_radiomics.columns if ('diagnostics' not in col)] # texture features
keep_cols = [col for col in test_radiomics.columns if ('diagnostics' not in col) and ('shape' in col)] # shape features only
test_radiomics = test_radiomics[keep_cols]  

# add relevant suffix to each column name, acc. to the ROI (check results.json file used)
test_radiomics.columns = [col + '_MN' for col in test_radiomics.columns]
test_radiomics.head()

,original_shape_Elongation_MN,original_shape_Flatness_MN,original_shape_LeastAxisLength_MN,original_shape_MajorAxisLength_MN,original_shape_Maximum2DDiameterColumn_MN,original_shape_Maximum2DDiameterRow_MN,original_shape_Maximum2DDiameterSlice_MN,original_shape_Maximum3DDiameter_MN,original_shape_MeshVolume_MN,original_shape_MinorAxisLength_MN,original_shape_Sphericity_MN,original_shape_SurfaceArea_MN,original_shape_SurfaceVolumeRatio_MN,original_shape_VoxelVolume_MN
0,0.882899,0.677544,21.418932,31.612600,35.847491,34.526755,35.811842,35.939520,12603.097965,27.910721,0.773529,3385.781968,0.268647,12632.465950
1,0.749906,0.722168,11.140591,15.426585,18.218944,15.432077,16.814540,18.533195,1313.943798,11.568484,0.789805,734.539673,0.559034,1326.330505
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.186012,0.071319,12.977519,181.963432,54.564487,111.954899,17.738453,120.251544,3047.770138,33.847334,0.525335,1935.094994,0.634922,3088.072174
4,0.362527,0.093074,13.834509,148.640061,115.558216,57.824780,75.726209,122.147341,9649.470493,53.886009,0.537360,4079.003413,0.422718,9716.795495


In [21]:
results_json = json.load(open('D:\\Shruti\\IS_project\\models\\mediastinal_nodes_volume\\results.json'))
selected_features = results_json['selected_features']

# make new dataframe X_test from test_radiomics and retain patient_id_NSCLC column
X_test = test_radiomics.copy()

# filter only texture features (remove shape features)
f_names = X_test.columns.tolist()
shape_features = [f for f in f_names if 'shape' in f]
X_test = X_test[shape_features]

# scale test data using the same scalar used for training
with open('D:\\Shruti\\IS_project\\models\\mediastinal_nodes_volume\\scalar.pkl', 'rb') as f:
    scaler = pickle.load(f)
X_test = pd.DataFrame(scaler.transform(X_test.values), columns=X_test.columns)  


# select only the selected features from test_radiomics
X_test_selected = X_test[selected_features]

# check if the number of selected features matches the number of columns in X_test_selected
print(len(selected_features), len(X_test_selected.columns))

12 12


c:\Users\P70073858\.conda\envs\shruti_env\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
# load CoxPH model from pickle file


with open('D:\\Shruti\\IS_project\\models\\mediastinal_nodes_volume\\coxph_model.pkl', 'rb') as f:
    cph = pickle.load(f)

# predict risk scores for test set
risk_scores = -cph.predict_partial_hazard(X_test_selected)

# calculate C-index --> please ensure you pass the correct survival time and event data for the test set patients here
test_c_index = concordance_index(event_observed=survival_time_column,  # --> edit as needed
                                 predicted_scores=risk_scores,
                                 event_times=event_column) # --> edit as needed
print("C-index on test set: ", test_c_index)

NameError: name 'survival_time_column' is not defined